# Model Optimization Attempts Using `keras_tuner`

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint
# Import the kerastuner library
import keras_tuner as kt
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import preproc_funcs as pf
import pandas as pd 

In [2]:
df = pf.intensive_preprocessing()

C:\Users\jonat\UO_Bootcamp\Mod_19\git_Neural_Network_Charity_Analysis\Neural_Network_Charity_Analysis\preproc_funcs.py:67: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  model_data_df = model_data_df.drop(final_cat,1)


In [6]:
# Split our preprocessed data into our features and target arrays
y = df.IS_SUCCESSFUL.values
X = df.drop(["IS_SUCCESSFUL"],1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

C:\Users\jonat\AppData\Local\Temp\ipykernel_3100\1724628131.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  X = df.drop(["IS_SUCCESSFUL"],1).values


In [15]:
# Create a method that creates a new Sequential model with hyperparameter options
def create_model(hp):
    number_input_features = len(X_train[0])
    
    nn_model = tf.keras.models.Sequential()

    # Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation',['relu','tanh'])
    
    # Allow kerastuner to decide number of neurons in first layer
    nn_model.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=1,
        max_value=300,
        step=5), activation=activation, input_dim=number_input_features))

    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 5)):
        nn_model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=1,
            max_value=300,
            step=5),
            activation=activation))
    
    nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))
    
    # Allow kerastuner to decide which optimization function to use
    optimizers = hp.Choice('optimizer',['Adadelta','Adagrad', 'Adam', 'Adamax', 'FTRL', 'NAdam', 'RMSprop', 'SGD'])
    
    # Compile the model
    nn_model.compile(loss="binary_crossentropy", optimizer=optimizers, metrics=["accuracy"])
    
    return nn_model

In [16]:
tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=50,
    hyperband_iterations=5,
    overwrite=True)

In [17]:
tuner.search(X_train,y_train,epochs=50,validation_data=(X_test,y_test))

Trial 450 Complete [00h 02m 32s]
val_accuracy: 0.7314285635948181

Best val_accuracy So Far: 0.7325947284698486
Total elapsed time: 04h 12m 58s
INFO:tensorflow:Oracle triggered exit


In [1]:
# Get top three model hyperparameters and print the values
top_hyper = tuner.get_best_hyperparameters(3)
for param in top_hyper:
    print(param.values)

NameError: name 'tuner' is not defined

In [ ]:
# Evaluate the top three models against the test dataset
top_model = tuner.get_best_models(3)
for model in top_model:
    model_loss, model_accuracy = model.evaluate(X_test_scaled,y_test,verbose=2)
    print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")